# Intro
体渲染直接在3D空间采样射线上的多个位置来计算射线颜色，不同于rasterization需要透视变换。

两者都约定相机坐标下的相机在原点，并且相机坐标轴由$\vec{\mathbf{x}},\vec{\mathbf{y}},\vec{\mathbf{z}}$三个轴定义，其中$\vec{\mathbf{z}}$与相机面向的方向相反，$\vec{\mathbf{y}}$为相机向上的方向，$\vec{\mathbf{x}}=-\vec{\mathbf{z}}\times \vec{\mathbf{y}}$（右手定则）。

体渲染的核心在于采样射线$\{\mathbf{p}(t)=\mathbf{o}+t\mathbf{v}|t\geq 0\}$上的点，射线的颜色为点颜色的加权和：
$$
C(\mathbf{o},\mathbf{v})=\int_0^{+\inf}w(t)c(\mathbf{p}(t),\mathbf{v})\mathrm{d}t,
$$
其中$w(t)$为权重函数。

在NeRF原文中，此权重函数$w(t)=T(t)\sigma(t)$。$T(t)=\exp(-\int_0^t\sigma(u)\mathrm{d}u)$

# 体渲染相机变换
<img src="../pasteImage/volume_rendering_world_camera.png">

### 旋转
旋转过程的目的是将世界坐标转化为相机坐标，$\mathbf{R}_{w2c}$的推导详见3D/viewport_tranformation.ipynb。最终的投影结果为：$\mathbf{K}\mathbf{R}_{w2c}$。

### 投影

此时$\mathbf{P}(x,y,z)$为camera coordinate下的坐标，$f$为focal length，因为$\mathbf{p}(u,v)$和$\mathbf{P}(x,y,z)$在一条线上，所以满足
$$
\begin{equation}
u=f\frac{x}{z}\quad v=f\frac{y}{z}
\end{equation}
$$


因为像素坐标是原点在边角，所以需要平移原点：
$$
\begin{equation}
u=f\frac{x}{z}+d_u\quad v=f\frac{y}{z}+d_v
\end{equation}
$$

假设真实的画布长宽$(H\times W)$，我们不妨用$Hu$和$Wv$代替$u\in\mathbb{R}$和$v\in\mathbb{R}$，此时$u,v\in\mathbb{N}$：
$$
\begin{equation}
Hu=f\frac{x}{z}+d_u\quad Wv=f\frac{y}{z}+d_v
\end{equation}
$$

等价于：
$$
\begin{equation}
u=\alpha_x\frac{x}{z}+d_x\quad y=\alpha_y\frac{y}{z}+d_y
\end{equation}
$$

整理成矩阵形式：
$$
\begin{equation}
z\mathbf{p}=
z\underbrace{\begin{pmatrix}
u\\
v\\
1
\end{pmatrix}}_{\mathbf{p}}
=\underbrace{\begin{pmatrix}
\alpha_x & 0 & d_x\\
0 & \alpha_y & d_y\\
0 & 0 & 1 \\
\end{pmatrix}}_{\mathbf{K}}
\underbrace{
\begin{pmatrix}
x\\
y\\
z
\end{pmatrix}}_{\mathbf{P}}
\end{equation}
$$

由于传感器没有垂直于光轴，传感器轴之间可能发生倾斜，$\mathbf{K}$中加入了$s$来代表这样的倾斜：
$$
\begin{equation}
\mathbf{K}=
\begin{pmatrix}
\alpha_x & s & d_x\\
0 & \alpha_y & d_y\\
0 & 0 & 1 \\
\end{pmatrix}
\end{equation}
$$

其中$\mathbf{K}$又常称为camera intrinsic matrix

<!-- ### Ray生成

<img src="../pasteImage/ray_generate.png"> -->